# 构建一个简单的LLM应用程序，使用聊天模型和提示模板

在这个快速入门中，我们将向您展示如何使用LangChain构建一个简单的LLM应用程序。这个应用程序将把文本从英语翻译成另一种语言。这是一个相对简单的LLM应用程序——它只是一个单一的LLM调用加上一些提示。然而，这是开始使用LangChain的一个好方法——许多功能可以通过一些提示和一个LLM调用来构建！

阅读本教程后，您将对以下内容有一个高层次的了解：

- 使用[语言模型](/docs/concepts/chat_models)

- 使用[提示模板](/docs/concepts/prompt_templates)

- 使用[LangSmith](https://docs.smith.langchain.com/)调试和跟踪您的应用程序

让我们开始吧！

## 设置

### Jupyter Notebook

本教程和其他教程可能最方便在[Jupyter notebooks](https://jupyter.org/)中运行。在交互式环境中浏览指南是更好地理解它们的好方法。请参阅[此处](https://jupyter.org/install)了解安装说明。

### 安装

要安装LangChain，请运行：

<!-- HIDE_IN_NB
import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
    <CodeBlock language="bash">pip install langchain</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
    <CodeBlock language="bash">conda install langchain -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>
HIDE_IN_NB -->

In [ ]:
# | output: false

# %pip install langchain
# 或
# %conda install langchain -c conda-forge

有关更多详细信息，请参阅我们的[安装指南](/docs/how_to/installation)。

### LangSmith

您使用LangChain构建的许多应用程序将包含多个步骤和多个LLM调用的调用。
随着这些应用程序变得越来越复杂，能够检查链或代理内部究竟发生了什么变得至关重要。
最好的方法是使用[LangSmith](https://smith.langchain.com)。

在上面的链接注册后，请确保设置您的环境变量以开始记录跟踪：

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
export LANGSMITH_PROJECT="default" # 或任何其他项目名称
```

或者，如果在笔记本中，您可以使用以下方式设置它们：

```python
import getpass
import os

try:
    # 从.env文件加载环境变量（需要`python-dotenv`）
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="输入您的LangSmith API密钥（可选）："
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='输入您的LangSmith项目名称（默认="default"）：'
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass(
        prompt="输入您的OpenAI API密钥（如果使用OpenAI则必需）："
    )
```

## 使用语言模型

首先，让我们学习如何单独使用语言模型。LangChain支持许多不同的语言模型，您可以互换使用它们。有关使用特定模型的详细信息，请参阅[支持的集成](/docs/integrations/chat/)。

<!-- HIDE_IN_NB>
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs overrideParams={{openai: {model: "gpt-4o-mini"}}} />
HIDE_IN_NB -->

In [2]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

让我们首先直接使用模型。[ChatModels](/docs/concepts/chat_models)是LangChain [Runnables](/docs/concepts/runnables/)的实例，这意味着它们提供了一个标准接口来与它们交互。为了简单地调用模型，我们可以将一组[消息](/docs/concepts/messages/)传递给`.invoke`方法。

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("将以下内容从英语翻译成意大利语"),
    HumanMessage("你好！"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-32654a56-627c-40e1-a141-ad9350bbfd3e-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

:::提示

如果我们启用了LangSmith，我们可以看到此运行已记录到LangSmith，并可以查看[LangSmith跟踪](https://smith.langchain.com/public/88baa0b2-7c1a-4d09-ba30-a47985dde2ea/r)。LangSmith跟踪报告了[令牌](/docs/concepts/tokens/)使用信息、延迟、[标准模型参数](/docs/concepts/chat_models/#standard-parameters)（例如温度）以及其他信息。

:::

请注意，ChatModels接收[消息](/docs/concepts/messages/)对象作为输入，并生成消息对象作为输出。除了文本内容外，消息对象还传递对话[角色](/docs/concepts/messages/#role)并保存重要数据，例如[工具调用](/docs/concepts/tool_calling/)和令牌使用计数。

LangChain还支持通过字符串或[OpenAI格式](/docs/concepts/messages/#openai-format)的聊天模型输入。以下是等效的：

```python
model.invoke("你好")

model.invoke([{"role": "user", "content": "你好"}])

model.invoke([HumanMessage("你好")])
```

### 流式传输

由于聊天模型是[Runnables](/docs/concepts/runnables/)，它们提供了一个标准接口，包括异步和流式传输调用模式。这使我们能够从聊天模型流式传输单个令牌：

In [4]:
for token in model.stream(messages):
    print(token.content, end="|")

|C|iao|!||

您可以在[本指南](/docs/how_to/chat_streaming/)中找到有关流式传输聊天模型输出的更多详细信息。

## 提示模板

现在我们直接将一组消息传递给语言模型。这组消息来自哪里？通常，它是由用户输入和应用程序逻辑的组合构成的。此应用程序逻辑通常会获取原始用户输入并将其转换为准备传递给语言模型的消息列表。常见的转换包括添加系统消息或使用用户输入格式化模板。

[提示模板](/docs/concepts/prompt_templates/)是LangChain中的一个概念，旨在协助完成此转换。它们接收原始用户输入并返回准备传递给语言模型的数据（提示）。

让我们在这里创建一个提示模板。它将接收两个用户变量：

- `language`：要将文本翻译成的语言
- `text`：要翻译的文本

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "将以下内容从英语翻译成{language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

请注意，`ChatPromptTemplate`支持单个模板中的多个[消息角色](/docs/concepts/messages/#role)。我们将`language`参数格式化到系统消息中，并将用户`text`格式化到用户消息中。

此提示模板的输入是一个字典。我们可以单独使用此提示模板来查看它的作用

In [ ]:
prompt = prompt_template.invoke({"language": "意大利语", "text": "你好！"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

我们可以看到它返回了一个`ChatPromptValue`，其中包含两个消息。如果我们想直接访问消息，我们可以这样做：

In [7]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

最后，我们可以在格式化的提示上调用聊天模型：

In [8]:
response = model.invoke(prompt)
print(response.content)

Ciao!


:::提示
消息`content`可以同时包含文本和具有附加结构的[内容块](/docs/concepts/messages/#aimessage)。有关更多信息，请参阅[本指南](/docs/how_to/output_parser_string/)。
:::

如果我们查看[LangSmith跟踪](https://smith.langchain.com/public/3ccc2d5e-2869-467b-95d6-33a577df99a2/r)，我们可以准确地看到聊天模型接收到的提示，以及[令牌](/docs/concepts/tokens/)使用信息、延迟、[标准模型参数](/docs/concepts/chat_models/#standard-parameters)（例如温度）以及其他信息。

## 结论

就是这样！在本教程中，您已经学习了如何创建您的第一个简单LLM应用程序。您已经学习了如何使用语言模型、如何创建提示模板以及如何通过LangSmith获得对您创建的应用程序的良好可观察性。

这只是您想要学习以成为熟练的AI工程师的冰山一角。幸运的是——我们还有很多其他资源！

有关LangChain核心概念的进一步阅读，我们有详细的[概念指南](/docs/concepts)。

如果您对这些概念有更具体的问题，请查看以下部分的操作指南：

- [聊天模型](/docs/how_to/#chat-models)
- [提示模板](/docs/how_to/#prompt-templates)

以及LangSmith文档：

- [LangSmith](https://docs.smith.langchain.com)